# Metody obliczeniowe w nauce i technice
## Laboratorium 4
### Singular Value Decomposition
#### Mateusz Surjak 

## Zadanie 1
Przygotuj duży (> 1000 elementów) zbiór dokumentów tekstowych w języku angielskim (np. wybrany korpus tekstów, podzbiór artykułów Wikipedii, zbiór dokumentów HTML uzyskanych za pomocą Web crawlera, zbiór rozdziałów wyciętych z
różnych książek)


In [2]:
import sys
from essential_generators import DocumentGenerator
import io
import re
import numpy as np
import nltk
nltk.download('punkt')
import os
import math

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\surja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Generator ktorym posłużyłem się przy generowaniu dokumentów, można odkomentować linię **'for j in range(1, 10):'** w celu wygenerowania większych plików

In [3]:
def gen_data():
    """
    Data generator
    """
    gen = DocumentGenerator()
    for i in range(1, 1101):
        with io.open(f"sample_text/text_{i}.txt", "w", encoding="utf-8") as f:
            # for j in range(1, 10):
            p = gen.paragraph()
            f.write(p)
            f.close()

In [4]:
# gen_data()

## Zadanie 2
Określ słownik słów kluczowych (termów) potrzebny do wyznaczenia wektorów
cech bag-of-words (indeksacja). Przykładowo zbiorem takim może być unia wszystkich słów występujących we wszystkich tekstach

In [5]:
def create_dict():
    dictionary={}
    for i in range(1, 1101):
        with io.open(f"sample_text/text_{i}.txt", "r", encoding="utf-8") as f:
            data = f.read()
            f.close()
            data = data.lower()
            data = re.sub(r'[^\w\s]','',data)
            data = re.sub('[0-9]','',data)
            tokenized_words=nltk.word_tokenize(data)
            for word in tokenized_words:
                if word in dictionary.keys():
                    dictionary[word]+=1
                else:
                    dictionary[word]=1
    return dictionary

In [6]:
dictionary = create_dict()
len(dictionary)

FileNotFoundError: [Errno 2] No such file or directory: 'sample_text/text_1.txt'

- Jak wydać w słowniku znajduje się ponad 14 tysięcy słów, może nie jest to wielka liczba ale znacznie przyspieszyła obliczenia.
Początkowo miałem ponad 130 tysięcy i python miał problemy z przetwarzaniem takich danych dlatego zdecydowałem się na zmniejszenie zawartości plików.

## Zadanie 3
Dla każdego dokumentu j wyznacz wektor cech bag-of-words $d_{j}$ zawierający częstości występowania poszczególnych słów (termów) w tekście.


In [100]:
def bag_of_words_vector(file):
    vector_dict={}
    for w in dictionary.keys():
        vector_dict[w]=0
    f = io.open(file,encoding="utf8")
    data = f.read()
    f.close()
    data = data.lower()
    data = re.sub(r'[^\w\s]','',data)
    data = re.sub('[0-9]','',data)
    words=nltk.word_tokenize(data)
    for w in words:
        vector_dict[w]+=1
    return vector_dict

Wektor cech będę przechowywał w postaci słownika gdyż przyspieszy to operacje odczytu. 

## Zadanie 4
Zbuduj rzadką macierz wektorów cech term-by-document matrix w której wektory cech ułożone są kolumnowo $A_{m×n}$ = $[d_{1}|d_{2}| . . . |d_{n}]$ (m jest liczbą termów w
słowniku, a n liczbą dokumentów)

In [101]:
def term_by_document_matrix():
    files = os.listdir('sample_text/')
    t_b_d_m = []
    for i in range(1, 1101):
        file_name = f'sample_text/text_{i}.txt'
        bow_vector=bag_of_words_vector(file_name)
        t_b_d_m.append(bow_vector)
    return t_b_d_m

In [102]:
term_by_document_mtx = term_by_document_matrix()

- Teraz w **term_by_document_mtx** znajdują się wektory dla każdego dokumentu.

## Zadanie 5
Przetwórz wstępnie otrzymany zbiór danych mnożąc elementy bag-of-words przez
inverse document frequency. Operacja ta pozwoli na redukcję znaczenia często występujących słów.
$$IDF(w) = log\frac{N}{n_{w}}$$
gdzie $n_{w}$ jest liczbą dokumentów, w których występuje słowo w, a N jest całkowitą
liczbą dokumentów.

In [103]:
def get_count_of_word(word):
    c = 0
    for i in term_by_document_mtx:
        if i.get(word)>0:
            c+=1
    return c

In [104]:
def multiply_by_IDF():
    for word in dictionary.keys():
        n_w = get_count_of_word(word)
        m = math.log(len(term_by_document_mtx)/n_w)
        for dic in term_by_document_mtx:
            dic[word] = dic.get(word)*m

In [105]:
multiply_by_IDF()


Teraz **term_by_document_mtx** została zmodyfikowana, każda jej wartość zostałą przemnożona przez IDF.
Pozwoliło to nadać większe znaczenie slowom które występują rzadko a zmniejszyć znaczenie słów występujących w wielu testach.


## Zadanie 6
Napisz program pozwalający na wprowadzenie zapytania (w postaci sekwencji
słów) przekształcanego następnie do reprezentacji wektorowej q (bag-of-words).
Program ma zwrócić k dokumentów najbardziej zbliżonych do podanego zapytania q. Użyj korelacji między wektorami jako miary podobieństwa
$$\cos{\theta_{j}} = \frac{q^{T}d_{j}}{||q|| ||d_{}j||}$$

In [106]:
def create_vector(data):
    vector_dict={}
    for w in dictionary.keys():
        vector_dict[w]=0
    data = data.lower()
    data = re.sub(r'[^\w\s]','',data)
    data = re.sub('[0-9]','',data)
    words=nltk.word_tokenize(data)
    for w in words:
        if w in vector_dict:
            vector_dict[w]+=1
    return list(vector_dict.values())

def get_closest_documents(data,k):
    data_vect = create_vector(data)
    holder = 0
    index = None
    to_return = []
    for idx,i in enumerate(term_by_document_mtx):
        cos = np.matmul(np.array(data_vect).T,np.array(list(i.values())))/(np.linalg.norm(data_vect)*np.linalg.norm(list(i.values())))
        if len(to_return)<k:
            to_return.append((idx,cos))
            to_return = sorted(to_return, key=lambda x: x[1])
        else:
            if cos > to_return[0][1] or math.isnan(to_return[0][1]):
                to_return[0] = (idx,cos)
                to_return = sorted(to_return, key=lambda x: x[1])
    return to_return

In [107]:
t = get_closest_documents("And extends industry awards, the juno awards, which were as important as the quilombo of. Was down and hemp plantations. these men.. Spaces is terrain. unlike most mammals, when cats bring. History, 1815-1970. australia (6.4 percent), saudi. Verification of savannah from the swampland, were widespread during the nonbreeding. Monarch of classroom. in 2013, the beach handball world championships. News media regional airport, bert mooney airport and the plant and animal species. Kamerun. later, basin, red lodge, and whitefish mountain resort near libby whitefish. Fever were extreme emotional."
                          ,3)
def read_file(index):
    """
    Dodaje 1 bo pliki zaindeksowałem od 1 a tablica liczy się od 0 
    """
    with io.open(f"sample_text/text_{index+1}.txt", "r", encoding="utf-8") as f:
      
        print("______________________________________")
        print(f.read())
        print("______________________________________")

print(t)
for (idx,cos) in t:
    read_file(idx)

[(318, 0.08172827634973914), (702, 0.1519243455746266), (8, 0.6824081772328199)]
______________________________________
An old populism, that were. To language. golbery. with the implementation of classical music.. Portal climate oil deposits were formed in the. And are this definition. Or kingdoms, broadly to include certain types of. Are édith on seattle in 1941. this left an extensive. Kalahari desert river, then the world's. Though whether germans live abroad. jews are. Asia. colonialism plantations. these men, women and could not obtain identification and leave. Which winds morelos, who occupied. Most ancient including rainier beach, van asselt, rainier, and jefferson south of. Through e-mail. and 1940s..
______________________________________
______________________________________
Precipitating deck and whitefish mountain resort near red lodge. Festivals, colonial lakes account for news. By aristippus other types. for example, the. Hadron and 0.14%. 
 
 the world of coca-cola, fe

#### Wnioski: 
Jako dane wejsciowe zostal podany jeden z plików, dokłądnie plik 9(w tablicy wiświetla się 8 ale to przez to że jest indeksowana od 0, a ja pliki zaindeksowałem od 1).
<br>
<br>
Dopasowanie do oryginalnego pliku wynosi 0,68, jest to spowodowane IDF. Gdy wyłączę IDF (zakomentuję wywołanie funkcji multiply_by_IDF()) kilka ramek wyżej to dopasowanie do orginalnego pliku będzie wynosić 1,0.

<br>
<br>
Reszta znalezionych plików moim zdaniem jest poprawna, wiele słów się pokrywa z wprowadzonym tesktem.

## Zadanie 7
Zastosuj normalizację wektorów cech $d_{j}$ i wektora q, tak aby miały one długość 1.
Użyj zmodyfikowanej miary podobieństwa otrzymując
$$ |q^{T}A| = [|\cos{\theta_{1}}|,|\cos{\theta_{2}}|,...,|\cos{\theta_{n}}|]$$


In [108]:
from sklearn.preprocessing import normalize

In [109]:
def normalize_matrix():
    to_return = []
    for idx,i in enumerate(term_by_document_mtx):
        to_return.append(normalize([list(i.values())],norm="l1"))
    return to_return
    

In [110]:
normalized_A = normalize_matrix()

- **normalized_A** jest w tym monemcie macierzą wektorów domumentów dzie każdy wektor ma długość 1.

In [111]:
def get_cos_vector(data):
    data_vector_normalized = normalize([create_vector(data)],norm="l1")
    to_return = []
    for idx,i in enumerate(normalized_A):
        cos = np.matmul(np.array(data_vector_normalized[0]).T,np.array(i[0]))/(np.linalg.norm(data_vector_normalized)*np.linalg.norm(i))
        to_return.append((cos,idx))
    return to_return

    

- Powyższa funkcja zwraca wektor podobieństw danych wejściowych do kolejnych dokumentów.

In [112]:
data = "And extends industry awards, the juno awards, which were as important as the quilombo of. Was down and hemp plantations. these men.. Spaces is terrain. unlike most mammals, when cats bring. History, 1815-1970. australia (6.4 percent), saudi. Verification of savannah from the swampland, were widespread during the nonbreeding. Monarch of classroom. in 2013, the beach handball world championships. News media regional airport, bert mooney airport and the plant and animal species. Kamerun. later, basin, red lodge, and whitefish mountain resort near libby whitefish. Fever were extreme emotional. "
def get_closest_documents_normalized(data,k):
    cos_vector = get_cos_vector(data)
    cos_vector = sorted(cos_vector, reverse=True)
    return cos_vector[:k]

In [113]:
cld = get_closest_documents_normalized(data,3)
print(cld)
for doc,i in cld:
    read_file(i)

[(0.6824081772328199, 8), (0.15192434557462658, 702), (0.08172827634973913, 318)]
______________________________________
And extends industry awards, the juno awards, which were as important as the quilombo of. Was down and hemp plantations. these men.. Spaces is terrain. unlike most mammals, when cats bring. History, 1815-1970. australia (6.4 percent), saudi. Verification of savannah from the swampland, were widespread during the nonbreeding. Monarch of classroom. in 2013, the beach handball world championships. News media regional airport, bert mooney airport and the plant and animal species. Kamerun. later, basin, red lodge, and whitefish mountain resort near libby whitefish. Fever were extreme emotional.
______________________________________
______________________________________
Precipitating deck and whitefish mountain resort near red lodge. Festivals, colonial lakes account for news. By aristippus other types. for example, the. Hadron and 0.14%. 
 
 the world of coca-cola, feat

### Wniosek:

- Dokumenty które otrzymaliśmy są identyczne kilka ramek wyżej, ich dopasowanie też jest takie samo jak przed normalizacją.

## Zadanie 8
W celu usunięcia szumu z macierzy A zastosuj SVD i low rank approximation

In [114]:
import scipy
def svd_and_low_rank_approx(k):
    A = []
    for a in normalized_A:
        A.append(a[0])
    u,s,vt = scipy.sparse.linalg.svds(np.array(A),k=k)
    return u @ np.diag(s)@vt


- Powyższa funkcja stosuje SVD i robi low_rank_approximation, zastosowałem bibliotekę scipy.

In [115]:
def get_after_approx(data,k):
    result = svd_and_low_rank_approx(k)
    data_vector_normalized = normalize([create_vector(data)],norm="l1")
    to_return = []
    for idx,i in enumerate(result):
        cos = np.matmul(np.array(data_vector_normalized[0]).T,np.array(i))/(np.linalg.norm(data_vector_normalized)*np.linalg.norm(i))
        to_return.append((cos,idx))
    return to_return

- Powyższa funkcja zwraca wektor podobieństw danych wejściowych do kolejnych dokumentów.

In [116]:
def get_closest_documents_after_approx(data,k,approx_k):
    cos_vector = get_after_approx(data,approx_k)
    cos_vector = sorted(cos_vector, reverse=True)
    return cos_vector[:k]


In [117]:
docs = get_closest_documents_after_approx(data,3,3)
print(docs)
for cos,idx in docs:
    read_file(idx)

[(0.23319098038557473, 718), (0.23319079346987068, 636), (0.23318859510505982, 459)]
______________________________________
In shared moral or ethical problems that is fallible by. In contrast, by coat type, or commonly as random-bred, moggies (chiefly british), or. That follows vivid descriptions of desert is generally accepted as coming from diverse. Moravec and march 27, 1964, the massive good friday. Medical specialties 6 vols. 
 
 
 
 despite its simplicity compared with classical. Of european-american old bridge. Part egyptian her arms (from a. Party strength 397 murders.) jens ludwig, director of works," from αρχι- (arkhi.. By cordell also faced criticism. Notaries, they alongside low german, sorbian, romany, and frisian; they are easy to. Four, having sudan. as of january 2017 the fertility rate is 1.73 children. Started in kingdom, norway.. About 50.1 religions include. Figures may of consciousness..
______________________________________
______________________________________

### Wniosek 1:
Dla małych wartości **approx_k** dokumenty zwrócone przez funckję nie są zbytnio podobne do dokumentu wejściowego.

In [118]:
docs = get_closest_documents_after_approx(data,3,200)
print(docs)
for cos,idx in docs:
    read_file(idx)

[(0.3192510838510946, 8), (0.2260017789480967, 508), (0.2171433751824727, 702)]
______________________________________
And extends industry awards, the juno awards, which were as important as the quilombo of. Was down and hemp plantations. these men.. Spaces is terrain. unlike most mammals, when cats bring. History, 1815-1970. australia (6.4 percent), saudi. Verification of savannah from the swampland, were widespread during the nonbreeding. Monarch of classroom. in 2013, the beach handball world championships. News media regional airport, bert mooney airport and the plant and animal species. Kamerun. later, basin, red lodge, and whitefish mountain resort near libby whitefish. Fever were extreme emotional.
______________________________________
______________________________________
Are jon emotion. nonverbal communication demonstrates one of 11 festivals with. And leaders major hub for many british cities. europe's population may fall. Summit of past few decades, most trucks will be d

### Wniosek 2:
Dla coraz większych wartości **approx_k** dokumenty coraz bardziej przepominają dokument wejściowy, dopasowanie jest bardzo dobre

## Zadanie 9
Porównaj działanie programu bez usuwania szumu i z usuwaniem szumu. Dla jakiej wartości k wyniki wyszukiwania są najlepsze (subiektywnie). Zbadaj wpływ
przekształcenia IDF na wyniki wyszukiwania.


### Porównaj działanie programu bez usuwania szumu i z usuwaniem szumu.
- Moim zdaniem program z usuwaniem szumu zwraca bardziej dopasowane wyniki, lecz K musi być odpowiednio duże gdyż dla małych k < 30 napewno można zneleźć lepiej dopasowane dokumenty.

### Dla jakiej wartości k wyniki wyszukiwania są najlepsze (subiektywnie).
- Jeśli zależy nam na dobrym wyniku to najlepiej dać większe k(approx_k w programie), lecz dla k większych niż 150 wyniki są naprawdę bardzo dobre dlatego szacuję że najlepszą wartością będzie około 170. 

### Zbadaj wpływ przekształcenia IDF na wyniki wyszukiwania.
- Często pojawiające się słowa w wielu dokumentach tracą swoje znaczenie na rzecz słów które pojawiają się w niewielu dokumentach. Pozwala to na lepsze dopasowanie dokumentów gdyż uniezależniamy wejściowy dokument od pospolitych słów które się często pojawiają i możemy skupić się na porównywaniu dokumentów po bardziej niszowych słowach.